## Convert .bin drawing to .npy drawing

### Import

In [1]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

### Load data

In [2]:
# Get random 1000 drawings from each object
def load_drawings(root, reshaped=False):
    all_files = glob.glob(os.path.join(root, '*.npy'))
    
    data = []
    for index, file in enumerate(all_files):
        np_arrays = np.load(file)
        if reshaped:
            new_arrays = []
            for idx in range(len(np_arrays)):
                np_arr = np.reshape(np_arrays[idx] ,(28,28,1))
                new_arrays.append(np_arr)
            np_arrays = new_arrays
        data.append(np_arrays)
    return data

In [3]:
data = load_drawings('npy_data', reshaped=True)

In [4]:
print(len(data[0]))

122876


In [5]:
def visualize(nparr):
    img = Image.fromarray(nparr.reshape(28,28))
    img.show(title="visualize array")

In [6]:
def set_lim(np_arrays, lim):
    lim_arr = []
    for arr in np_arrays:
        i = 0
        for item in arr:
            if i == lim:
                break
            lim_arr.append(item)
            i += 1
    return lim_arr

In [7]:
data = set_lim(data, 1000)

In [8]:
print(len(data))

3000


In [9]:
def normalize(nparr):
    return np.interp(nparr, [0, 255], [-1,1])

In [27]:
def denormalize(nparr):
    return np.interp(nparr, [-1, 1], [0, 255])

In [10]:
data = normalize(data)

### Labeling

In [11]:
object_file = open("object.txt", "r")
objects = object_file.readlines()
object_file.close()
N_CLASSES = len(objects)
CLASSES = {}
for idx, obj in enumerate(objects):
    CLASSES[idx] = obj.replace('\n', '')
print(CLASSES)


{0: 'triangle', 1: 'circle', 2: 'square'}


In [12]:
def make_label(n1, n2):
    labels = []
    for idx in range(n1):
        labels += [idx] * n2
    return labels

In [13]:
labels = make_label(N_CLASSES, 1000)

In [14]:
print(labels[1000:1010])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [15]:
from sklearn.model_selection import train_test_split as tts
from keras.utils import np_utils

In [16]:
x_train, x_test, y_train, y_test = tts(data, labels, test_size=0.1)

In [17]:
print(x_train[0])

[[[-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]]

 [[-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]]

 [[-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-1.        ]
  [-

In [18]:
Y_train = np_utils.to_categorical(y_train, N_CLASSES)
Y_test = np_utils.to_categorical(y_test, N_CLASSES)

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten


def conv(classes, input_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(classes, activation='softmax'))
    return model


In [20]:
model = conv(classes=N_CLASSES, input_shape=(28,28,1))

In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
EPOCHS = 20
model.fit(np.array(x_train), np.array(Y_train), batch_size=32, epochs=EPOCHS, verbose=1)

Epoch 1/20
85/85 [==============================] - 3s 24ms/step - loss: 0.2531 - accuracy: 0.9081
Epoch 2/20
85/85 [==============================] - 2s 22ms/step - loss: 0.0992 - accuracy: 0.9685
Epoch 3/20
85/85 [==============================] - 2s 20ms/step - loss: 0.0694 - accuracy: 0.9756
Epoch 4/20
85/85 [==============================] - 2s 23ms/step - loss: 0.0522 - accuracy: 0.9837
Epoch 5/20
85/85 [==============================] - 2s 23ms/step - loss: 0.0437 - accuracy: 0.9867
Epoch 6/20
85/85 [==============================] - 2s 20ms/step - loss: 0.0290 - accuracy: 0.9911
Epoch 7/20
85/85 [==============================] - 2s 21ms/step - loss: 0.0322 - accuracy: 0.9907
Epoch 8/20
85/85 [==============================] - 2s 27ms/step - loss: 0.0114 - accuracy: 0.9978
Epoch 9/20
85/85 [==============================] - 2s 23ms/step - loss: 0.0237 - accuracy: 0.9922
Epoch 10/20
85/85 [==============================] - 2s 28ms/step - loss: 0.0193 - accuracy: 0.9926
Epoch 11/

In [23]:
pred = model.predict(np.array(x_test))

10/10 [==============================] - 0s 7ms/step


In [24]:
score = 0

for i in range(len(pred)):
    if np.argmax(pred[i]) == y_test[i]:
        score+=1

acc = ((score+0.0)/len(pred)*100)
print("Accuracy: {acc}".format(acc = ((score+0.0)/len(pred)*100)))

Accuracy: 97.33333333333334


In [26]:
model.save("model/doodle.h5")

In [30]:
from random import randint
def visualize_and_predict():
    "selects a random test case and shows the object, the prediction and the expected result"
    n = randint(0, len(x_test))
    visualize(denormalize(np.reshape(x_test[n], (28, 28))))
    pred = CLASSES[np.argmax(model.predict(np.array([x_test[n]])))]
    actual = CLASSES[y_test[n]]
    print("Actual:", actual)
    print("Predicted:", pred)


In [31]:
visualize_and_predict()

1/1 [==============================] - 0s 34ms/step
Actual: circle
Predicted: circle
